This challenge is inspired by this cipher https://www.cryptogram.org/downloads/aca.info/ciphers/Condi.pdf 

**_"Easy"_ solution:** Set up the constraints and feed it to a SAT solver (I totally saw this coming...). There are many ways to go about this challenge. See below for an alternative and fun, but definitely over-engineered and unneccessarily complicated linear algebra approach. If the group operation was something more complicated than just XOR, this approach might've been better.

### Cipher

The key, $K$, is just a permutation of the alphabet, where the alphabet is just the hexadecimal characters.

The size of the key space is `16!` which is infeasible to bruteforce, but we can recover the key(s) given enough plaintext.

**Notation:**

- Let $K_i$ denote the letter at index $i$ of the key.
- Let $K(m)$ denote the index of the character $m$ in the key.

For example if the key is `b4536f8d0a729ce1`, then $K_0$ is `'b'` and $K(\texttt{e})$ is `14`.

Notice that $K(K_i) = i$ and $K_{K(i)} = i$.

**Addition:**

The peculiar `add` method operates on 4 bit numbers and adds the numbers bit wise modulo 2. This is just XOR. Recognising this as addition in $\mathrm{GF}(16)$ which will come in handy later on. We'll denote this addition operation by $\star$.

Note that $x \star x = 0$, that is, each element is its own inverse.

**Encryption:**

Let $m$ be the message we want to encrypt. Then, let

$$e_i = \begin{aligned} \begin{cases} K(m_i) \star s \quad & i = 0 \\ K(m_i) \star K(m_{i-1}) \quad & i > 0 \end{cases} \end{aligned}$$

Then $c_i = K_{e_i}$

**Decryption:**

This isn't given in the challenge, but it's easy enough to figure out.

Let

$$d_i = \begin{aligned} \begin{cases} K(c_i) \star s \quad & i = 0 \\ K(c_i) \star K(m_{i-1}) \quad & i > 0 \end{cases} \end{aligned}$$

Alternatively, using the identity $K(K_i) = i$, we get

$$d_i = \begin{aligned} \begin{cases} e_i \star s \quad & i = 0 \\ e_i \star K(m_{i-1}) \quad & i > 0 \end{cases} \end{aligned}$$

Then $m_i = K_{d_i}$

**Proof of correctness:**

For $i = 0$

$$\begin{aligned} d_0 &= e_0 \star s \\ &= K(m_0) \star s \star s \\ &= K(m_0) \end{aligned}$$

For $i > 0$ the decryption gives

$$\begin{aligned} d_i &= e_i \star K(m_{i-1}) \\ &= K(m_i) \star K(m_{i-1}) \star K(m_{i-1}) \\ &= K(m_i) \end{aligned}$$

In both cases, $K_{d_i} = K_{K(m_i)} = m_i$.

### Analysis

Our goal will be to set up constraints on the key. This should be easy since we're given some plaintext, and looking at the encryption/decryption algorithms, we can see that the plaintext and ciphertext are very closely related.

Let $P_i$ be the $i$th character of known plaintext, and let $C_i$ be the corresponding ciphertext character. Then

$$K(C_i) = K(P_i) \star K(P_{i-1})$$

so

$$K(C_i) \star K(P_{i-1}) \star K(P_i) = 0$$

We'll use this to set up a system of linear equations that we can easily solve.

Let $\mathbf{e}_i$ (defined for $i > 0$) be a vector of size 16 whose entries are in $\mathrm{GF}(16)$. Set all entries to $0$, then add $1$ at the positions given by $C_i$, $P_{i-1}$ and $P_i$.

For example, if $C_1 = \texttt{0}, P_0 = \texttt{a}, P_1 = \texttt{3}$, then $\mathbf{e}_1 = (1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0)$.

Let $\mathbf{k} = (k_1, k_2, \ldots, k_{16})$ be a vector that represents the key. In particular $k_i$ represents the position in the key of the $i$th character in the alphabet.

For example, if the key is `b4536f8d0a729ce1`, then $\mathbf{k} = (8,15,11,3,1,2,4,10,6,12,9,0,13,7,14,5)$.

Now let $M$ be the matrix whose rows are the $\mathbf{e}_i$. Then $M \cdot \mathbf{k} = 0$. So to solve for $\mathbf{k}$, we search vectors in the null space of $M$. There will potentially be a lot of vectors that satisfy these constraints. Since we're given more than 16 pairs of plaintext/ciphertext characters, we can try each set of 16 to find one with a low enough nullity to bruteforce.

```python
from os import urandom
from random import choices, shuffle
from itertools import product
from tqdm import tqdm

ALPHABET = '0123456789abcdef'
N = len(ALPHABET)

plaintext = b'The secret message is:'.hex()
ciphertext = '85677bc8302bb20f3be728f99be0002ee88bc8fdc045b80e1dd22bc8fcc0034dd809e8f77023fbc83cd02ec8fbb11cc02cdbb62837677bc8f2277eeaaaabb1188bc998087bef3bcf40683cd02eef48f44aaee805b8045453a546815639e6592c173e4994e044a9084ea4000049e1e7e9873fc90ab9e1d4437fc9836aa80423cc2198882a'

class Cipher:
    def __init__(self, key):
        self.key = key
        self.n = len(self.key)
        self.s = 7

    def add(self, num1, num2):
        res = 0
        for i in range(4):
            res += (((num1 & 1) + (num2 & 1)) % 2) << i
            num1 >>= 1
            num2 >>= 1
        return res

    def encrypt(self, msg):
        key = self.key
        s = self.s
        ciphertext = ''
        for m_i in msg:
            c_i = key[self.add(key.index(m_i), s)]
            ciphertext += c_i
            s = key.index(m_i)
        return ciphertext

    def decrypt(self, ciphertext):
        key = self.key
        s = self.s
        plaintext = ''
        for c_i in ciphertext:
            m_i = key[self.add(key.index(c_i), s)]
            plaintext += m_i
            s = key.index(m_i)
        return plaintext

#### SOLUTION

def num_to_gf16_poly(num):
    b = []
    for _ in range(4):
        b.append(num & 1)
        num >>= 1
    return GF(16)(b)

def gf16_to_num(poly):
    return poly.integer_representation()

def get_eqn_matrix(known_pt, known_ct, n=len(ALPHABET)):
    M = []

    L = known_ct[1:]
    S = known_pt
    R = known_pt[1:]

    for l,s,r in zip(L, S, R):
        eqn = [0]*n
        eqn[ALPHABET.index(l)] += num_to_gf16_poly(1) 
        eqn[ALPHABET.index(s)] += num_to_gf16_poly(1)
        eqn[ALPHABET.index(r)] += num_to_gf16_poly(1)
        M.append(eqn)

    return M

def vec_to_key(vec, n=N):
    key = ['0']*n
    for i,v in enumerate(vec[:n]):
        key[gf16_to_num(v)] = ALPHABET[i]
    return ''.join(key)

for i in range(1, len(plaintext)-N):
    M = get_eqn_matrix(plaintext, ciphertext)[i:i+N]
    M = Matrix(GF(16), M)
    nullity = M.nullity()

    if nullity > 5: # too much to bruteforce
        continue

    B = M.right_kernel_matrix()

    for lc in tqdm(product(range(N), repeat=nullity), total=N^nullity):
        v = B.linear_combination_of_rows([num_to_gf16_poly(alpha) for alpha in lc])

        if len(set(v)) != len(v):
            continue

        key = vec_to_key(v)
        cipher = Cipher(key)
        d = bytes.fromhex(cipher.decrypt(ciphertext))

        if b'DUCTF' in d:
            print(d)
```